###### Grabbing our file

In [1]:
import os.path
import io
save_path = "C:/Users/avish/OneDrive/Documents/Presidential-Transcripts/Files"

In [2]:
import re
import pandas as pd
import numpy as np

###### Administration

In [3]:
def getadmin(text):
    administration = re.findall('(?<=Administration of ).*?(?=,)', text)
    if len(administration) == 0:
        administration = re.findall('William J. Clinton', text)
        if len(administration) == 0:
            administration = re.findall('George W. Bush', text)
            if len(administration) == 0:
                administration = re.findall('George Bush', text)
                if len(administration) == 0:
                    administration = re.findall('Bill Clinton', text)[0]
                else:
                    administration = administration[0]
            else:
                administration = administration[0]
        else:
            administration = administration[0]
    else:
        administration = administration[0]
    return administration

###### Number

In [4]:
def getnumber(text):
    number = re.findall('(?<=Proclamation ).*?(?=\D)', text)
    if len(number) == 0:
        number = re.findall('Proclamation--', text)
        if len(number) == 0:
            return np.nan
        else:
            number = number[0]
        number = re.sub('Proclamation', '', number)[0]
    else:
        number = number[0]
    return number

###### Title

In [5]:
def gettitle(text):
    text = re.sub('\n\n', 'SPLIT-HERE', text)
    badtitle = re.findall('Proclamation ?[\d-][\w\W]*?SPLIT-HERE', text)
    if len(badtitle) == 0:
        badtitle = re.findall('Proclamation  ?\d[\w\W]*?<\/b>', text)
        if len(badtitle) == 0:
            text = re.sub('SPLIT-HERE', '\n\n', text)
            return np.nan
        title = re.sub('â€”', ' - ', badtitle[0])
        title = re.sub('\n' ,'', title)
        title = re.sub(' <\/b>', '', title)
    else:
        title = re.sub('â€”', ' - ', badtitle[0])
        title = re.sub('SPLIT-HERE', '', title)
        title = re.sub('\n' ,'', title)
    text = re.sub('SPLIT-HERE', '\n\n', text)
    return title

###### Date

In [6]:
def getdate(text):
    date = re.findall("(?<=p>\\n<i>\\n ).*?(?= ) [\d].*?, \d\d\d\d", text)
    if len(date) == 0:
        date = re.findall('\n\n.*?\n\n', text)
        if len(date) == 0:
            date = re.findall('[A-Z][a-z]*? \d\d?, \d\d\d\d', text)[0]
        else:
            date = date[0]
        date = re.sub('\n\n', '', date)
        if date[0] == ' ':
            date = date[1:]
    else:
        date = date[0]
    return date

###### Date Filed

In [7]:
def getfiled(text):
    filed = re.findall('Filed with t?h?e? ?Office of .*\n*', text)
    if len(filed) == 0:
        return np.nan
    else:
        filed = filed[0]
    filed = re.findall('\d.*', filed)[0]
    filed = re.sub(',', '', filed)
    filed = re.sub(']', '', filed)
    return filed

###### Note

In [8]:
def getnote(text):
    note = re.findall('Note[: ] ?[^.]*', text)
    if len(note) == 0:
        note = re.findall('N\n <font size="\+1">\n  OTE\n  <font size="\+1">\n.*', text)
        note = re.findall(':(.*)', note[0])[0]
        return note
    note = re.sub('\\n|<i>|<\/i>', '', note[0])
    note = re.sub('       |      ', ' ', note)
    note = note + '.'
    return note

###### Categories

In [9]:
def getcategories(text):
    categories = re.findall('(?<=Categories.[\n] <\/i>\n ).*(?=\n)', text)
    if len(categories) == 0:
        categories = np.nan
    else:
        categories = categories[0]
        categories = re.sub('[Pp]roclamations : ', '', categories)
    return categories


###### Subjects

In [10]:
def getsubjects(text):
    subjects = re.findall('Subjects.*\n.*\n.*', text)
    if len(subjects) == 0:
        subjects = np.nan
    else:
        subjects = subjects[0]
        subjects = re.sub('\n', '', subjects)
        subjects = re.sub('<\/i>', '', subjects)
        subjects = re.sub('  ', ' ', subjects)
        subjects = re.sub('[Ss]ubjects: ', '', subjects)
    return subjects

###### CPD Number

In [11]:
def getcpdnum(name):
    cpd = re.findall('(?<=DCPD).*?(?=\.)', name)
    if len(cpd) == 0:
        cpd = re.findall('WCPD\d*', name)[0]
        cpd = re.sub('WCPD', '', cpd)
    else:
        cpd = cpd[0]
    return cpd

Test if it's a proclamation

In [12]:
def isproc(text):
    re.findall('[Aa] [Pp]roclamation\n', text)[0]

###### Getting all of our proclamations

In [ ]:
proclamations = []
names = []
for file in os.listdir(save_path):
    name = os.path.join(save_path, file)
    try:
        with open(name, "r") as f:
            text = f.read()
    except:
        pass
    try:
        isproc(text)
        proclamations.append(text)
        names.append(file)
    except:
        pass

In [23]:
coltitle = []
colcpd = []
colsubj = []
colcateg = []
colnote = []
colfiled = []
coldate = []
colnum = []
coladmin = []
for text in proclamations:
    print(text)
    break
    coltitle.append(gettitle(text))
    colsubj.append(getsubjects(text))
    colcateg.append(getcategories(text))
   # colnote.append(getnote(text))
    colfiled.append(getfiled(text))
    coldate.append(getdate(text))
    colnum.append(getnumber(text))
    coladmin.append(getadmin(text))
for name in names:
    break
    colcpd.append(getcpdnum(name))

In [24]:
proclamationstable = pd.DataFrame()
proclamationstable['Title'] = coltitle
proclamationstable['Date'] = coldate
proclamationstable['Num'] = colnum
proclamationstable['Filed'] = colfiled
proclamationstable['Subject'] = colsubj
proclamationstable['Category'] = colcateg
proclamationstable['CPD Number'] = colnum
proclamationstable['Administration'] = coladmin

In [25]:
proclamationstable

,Title,Date,Num,Filed,Subject,Category,CPD Number,Administration
0,"Proclamation 6522--Braille Literacy Week, 1993","January 5, 1993",6522,2:28 p.m. January 6,NaN,NaN,6522,George Bush
1,Proclamation 6523--National Law Enforcement Tr...,"January 5, 1993",6523,3:57 p.m. January 6,NaN,NaN,6523,George Bush
2,"Proclamation 6520--National Good Teen Day, 1993","December 23, 1992",6520,4:31 p.m. January 4,NaN,NaN,6520,George Bush
3,Proclamation 6521--National Sanctity of Human ...,"January 4, 1993",6521,4:32 p.m. January 4,NaN,NaN,6521,George Bush
4,"Proclamation 6524--Martin Luther King, Jr., Fe...","January 11, 1993",6524,11:27 a.m. January 12,NaN,NaN,6524,George Bush
5,Proclamation 6525--National Day of Fellowship ...,"January 20, 1993",6525,11:33 a.m. January 21,NaN,NaN,6525,William J. Clinton
6,Proclamation 6526--Death of Thurgood Marshall,"January 24, 1993",6526,1:45 a.m. January 25,NaN,NaN,6526,William J. Clinton
7,Proclamation 6527--National Women and Girls in...,"February 3, 1993",6527,2:11 p.m. February 4,NaN,NaN,6527,William J. Clinton
8,"Proclamation 6528--American Heart Month, 1993","February 14, 1993",6528,11:11 a.m. February 16,NaN,NaN,6528,William J. Clinton
9,Proclamation 6529--National Visiting Nurse Ass...,"February 18, 1993",6529,12:05 p.m. February 19,NaN,NaN,6529,William J. Clinton


In [27]:
proclamationstable.to_csv('proclamations.csv')